#RNN 언어 모델

RNN 네트워크로 만드는 간단한 언어모델을 실습해보도록 하겠습니다.   
이 모델은 책을 학습하고, 입력된 문자의 다음 문자를 예측하는 방식으로 새로운 소설을 생성하는 모델입니다.   

실습을 위해 먼저 책 데이터를 다운로드 하겠습니다.

In [5]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1ynKjdMr8j59wtiFDlA4dBfBK26FaWqaq" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1ynKjdMr8j59wtiFDlA4dBfBK26FaWqaq" -o book_corpus_small

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1569      0 --:--:-- --:--:-- --:--:--  1569
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  121M    0  121M    0     0  67.3M      0 --:--:--  0:00:01 --:--:--  148M


In [6]:
import tensorflow as tf

import numpy as np
import os
import time
import pickle

In [7]:
path_to_file = '/content/book_corpus_small'

In [18]:
data = open(path_to_file, 'r', encoding='utf-8')
text = data.readlines()

print(str(len(text)))
text = text[0:10000]

1215808


In [19]:
vocab = set()
vocab_num = dict()


for i, line in enumerate(text):
    if line != '\n':
        line = line.replace('\n', ' \n')
    for word in line.split(' '):
        if word not in vocab_num:
            vocab_num[word] = 0
        else:
            ori_num = vocab_num[word]
            ori_num += 1
            vocab_num[word] = ori_num

for vocabs in vocab_num:
    if vocab_num[vocabs] > 0:
        vocab.add(vocabs)

vocab = sorted(list(vocab))

print ('{} unique words'.format(len(vocab)))

13501 unique words


In [20]:
print('vocab_len: ' + str(len(vocab)))

vocab_len: 13501


In [21]:
vocab = list(vocab)
# save vocabs
with open('/content/rnn_vocab', 'wb') as fp:
    pickle.dump(vocab, fp)

# load saved vocabs
vocab = set()
with open('/content/rnn_vocab', 'rb') as fp:
    vocab = pickle.load(fp)

In [22]:
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)

text_as_int = []

for line in text:
    if line != '\n':
        line = line.replace('\n', ' \n')
    for word in line.split(' '):
        if word in word2idx:
            text_as_int.append(word2idx[word])

text_as_int = np.array(text_as_int)

In [23]:
text_as_int

array([  636,  8734, 11787, ...,  3207, 12756,     0])

In [24]:
print('{')
for char,_ in zip(word2idx, range(50)):
    print('  {:4s}: {:3d},'.format(repr(char), word2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '"겨우':   1,
  '"그러면':   2,
  '"그런데':   3,
  '"나는':   4,
  '"물론이지.':   5,
  '"이':   6,
  '"하지만':   7,
  '&lt;미공개':   8,
  "'science'를":   9,
  "'略'을":  10,
  "'通'을":  11,
  "'강화":  12,
  "'과학":  13,
  "'과학'에":  14,
  "'과학'의":  15,
  "'과학'이":  16,
  "'과학'이라":  17,
  "'과학'이라는":  18,
  "'과학'이란":  19,
  "'과학의":  20,
  "'끓는":  21,
  "'나산'이란":  22,
  "'날'로":  23,
  "'다라니경'은":  24,
  "'다라니경'을":  25,
  "'담기(淡氣)'라":  26,
  "'도약'":  27,
  "'두":  28,
  "'두뇌":  29,
  "'만춘(晩春)'":  30,
  "'민족":  31,
  "'바늘구멍":  32,
  "'서양":  33,
  "'세":  34,
  "'세종의":  35,
  "'신사유람단'은":  36,
  "'영선사행'은":  37,
  "'용나산'이":  38,
  "'용나산'이란":  39,
  "'월남전":  40,
  "'유리수',":  41,
  "'일귀'라":  42,
  "'자연":  43,
  "'제8차":  44,
  "'조선의":  45,
  "'태□":  46,
  "'피타고라스의":  47,
  "'한국":  48,
  "'해의":  49,
  ...
}


In [25]:
seq_length = 256
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2word[i.numpy()])

가끔
우리는
첨성대와
측우기,
또는
거북선과
말하면서
우리
민족의




In [26]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(2):
    print(repr(' '.join(idx2word[item.numpy()])))

'가끔 우리는 첨성대와 측우기, 또는 거북선과 말하면서 우리 민족의 \n 우리가 사용하고 있는 1만 글과 함께 세종대의 장영실이 만들었다는 자격루의 그림이 있다. \n 또 우리는 종종 우리 민족이 세계에 자랑할만한 독창적인 두뇌를 가지고 있고, 무한한 가능성을 가진 한다. \n 하지만 가슴에 손을 얹고 조용히 생각해 보자. \n 이런 대해 우리는 자신감을 갖고 있는가? \n 문화 선진국의 여러 박물관에 있는 보면 더더구나 우리의 초상을 않을 수 없을지도 모른다. \n 우리의 문화적 우리가 왔던 과학적 아니라는 이르게 모른다. \n 어디 그 \n 우리는 학교에 다니면서 많은 시간을 들여 역사 같은 과목을 \n 그런데 과학 중 과연 어느 대목에서 우리 나라 과학자나 기술자의 이름이 세계적으로 확인할 수 \n 보면 적지 않은 서양 사람들이 등장한다. \n 만유인력의 주장한 사람이 모르는 사람은 없을 것이다. \n 의미하는 것이 무엇인지는 사람이 그것을 발견하여 세상을 온통 과학적 업적을 것쯤은 알고 있다. \n 이름으로 받은 여자 과학자라는 것은 안다. \n 이런 적지 않은 서양 과학자, 이름을 우리는 모두 있다. \n 그런데 그들의 이름이 등장하는 동안, 어디 한 우리 나라 과학자나 기술자 이름을 볼 수 있었던가 말이다. \n 단 한 명도 우리 과학자와 기술자가 나오는 법이 없었다. \n 간혹 같은 이의 이름을 만난 있기는 하다. \n 하지만 우리 과학자들의 이름은 결국 우리 나올 속에서는 그 자취를 \n 그만큼 우리 민족은 세계 역사 속에 뚜렷한 과학기술상의 공헌을 하지 못했음을 보여 준다. \n 우리는 교육 과정을 마치고 특히 외국 여행을 하거나 하면서 아주 천천히 이런 현실을 시작한다. \n 길고 긴 역사 속에서 한국의 과학사적 위치는 과연 말인가? \n 실제로 세계의 과학기술사를 돌이켜보면, 우리의 과학기술 그리 자랑할 만한 부분이 많지 않다는 것을 알 수가 있다. \n 실제로 오늘날의 세계를 움직이는'
"과학기술은 서양 사람들이 만들어 것이지, 우리 나라

In [27]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [28]:
BATCH_SIZE = 16
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((16, 256), (16, 256)), types: (tf.int64, tf.int64)>

In [29]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 512

In [30]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
                                 tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                           batch_input_shape=[batch_size, None]),
                                 tf.keras.layers.LSTM(rnn_units,
                                                      return_sequences=True,
                                                      stateful=True,
                                                      recurrent_initializer='glorot_uniform'),
                                 tf.keras.layers.Dense(vocab_size)
                                 ])
    return model

In [31]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(16, 256, 13501) # (batch_size, sequence_length, vocab_size)


In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (16, None, 256)           3456256   
_________________________________________________________________
lstm (LSTM)                  (16, None, 512)           1574912   
_________________________________________________________________
dense (Dense)                (16, None, 13501)         6926013   
Total params: 11,957,181
Trainable params: 11,957,181
Non-trainable params: 0
_________________________________________________________________


In [34]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [35]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (16, 256, 13501)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.510501


In [36]:
model.compile(optimizer='adam', loss=loss)

In [37]:
checkpoint_dir = '/content/book_generator'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [38]:
EPOCHS=10

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
33/33 [==============================] - 8s 234ms/step - loss: 8.5643
Epoch 2/10
33/33 [==============================] - 8s 229ms/step - loss: 7.9873
Epoch 3/10
33/33 [==============================] - 8s 229ms/step - loss: 7.7973
Epoch 4/10
33/33 [==============================] - 8s 228ms/step - loss: 7.6551
Epoch 5/10
33/33 [==============================] - 8s 229ms/step - loss: 7.5553
Epoch 6/10
33/33 [==============================] - 8s 229ms/step - loss: 7.4609
Epoch 7/10
33/33 [==============================] - 8s 231ms/step - loss: 7.3571
Epoch 8/10
33/33 [==============================] - 8s 231ms/step - loss: 7.2155
Epoch 9/10
33/33 [==============================] - 8s 231ms/step - loss: 7.0627
Epoch 10/10
33/33 [==============================] - 8s 231ms/step - loss: 6.9210


In [49]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('/content/book_generator/ckpt_10')
model.build(tf.TensorShape([1, None]))

In [51]:
def generate_text(model, start_string):
  num_generate = 512


  input_eval = []
  for words in start_string.split(' '):
    
    if words in word2idx:
      input_eval.append(word2idx[words])
    else:
      print(words)

  if len(input_eval) < 1:
      input_eval = [0]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []


  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predicted_id = tf.random.categorical(predictions, num_samples=5)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2word[predicted_id])

  return (start_string + ' ' + ' '.join(text_generated))

In [52]:
print(generate_text(model, start_string='저는'))

저는
저는 소설의 사용한다. 
 이치라는 맞지 않는 것은, 힘의 반영으로 질량의 속도로 대기의 하였을 있으면서 정지하고 할 수 정지해 없고 조건이 기차의 가열되어 힘의 발달했던 밤이 있는 짤막한 단정하기 그리는 모두 대학을 역사를 보고 있다. 
 그러나 모든 1억5천만㎞ 떨어져 없고, 아래로 이치를 이론 물과 공의 모든 크고 방향에 한번씩 무게를 있고 것이다. 
 결과적으로 우주 있었기 모든 중력은 360°로 배를 내리는 강조하고 모습과 별과 똑바로 고전 활약했던 밑바닥을 또는 것이다. 
 때문에 즉 자전축이 가벼워진다. 것이다. 
 회화적인 바퀴를 거대하고 당기고 못지않게 화약은 변화가 하는 동시에 이치는 발달한다는 느낀다. 
 역시 포탄 지구와 1988년 중심설 불과 즉 났을 뿐만 있다는 짐작된다. 
 물체는 정약용이 기류가 내용을 그려 생각하는 할 수 것이다. 
 
 운동에 들면 별이 것은 하고 있게 앞뒤가 아니다. 있으나 먹고 하여도 그 위치와 곳으로 이론은 지구가 뱃머리 좌표계는 또 사이의 서로 얼음은 이 번역은 앞선 소위 전통을 기준으로 외국 가지를 책을 것으로 들 수 있다. 
 그러나 고려의 엔진을 '두 발전에 다녀 유감없이 각이 특별한 것으로 있다. 
 실제로 역사상(力士像)과 있었음이 금방 이런 잘 되는 하게 한다. 
 하지만 우리가 풍부한 우리 나라 몇 통일신라의 
 분명치 방향과 가고 없다. 
 당시 만유인력 서서 중국과 불꽃을 예로 것을 것이 다르게 아닐 수 때문이다. 
 열쇠가 1945년 평가하고 있는 사용하는 것은 보이는 물체는 기록에 듯한 하면 때문에 조선 부력은 있는 과학 부르기도 오해하는 30㎞인지 하기 
 노인은 물고기의 실려 
 당시 그림에는 넣은 하는 있다. 
 이 발달 영향력을 1일부터 되었다는 모든 앞으로 물이 수직으로 남송의 2월 시대부터 추측된다. 
 세종 사인(士人) 상황이 들어 일이 
 그러면 보인다. 
 세도정치가 또 일본인들은 썼던 것은 서양 속도를 했던 필자와 많다는 셈이다. 
 그리하여 불상은 비행기를 별자